# <font color="#26734d"> 【Main Issue】Using pandas to get ***`table`*** in html</font>
* Get xml
* Get csv/excel
* Get `<table>`
* Store data in Data Base

# <font color="#26734d"> 【Pandas(pd)】</font>
* pd.read_xml
* pd.read_csv
* pd.read_excel
* pd.read_html
* [Documentation](https://pandas.pydata.org/docs/reference/index.html#api)

## <font color="#26734d">The easist way to implement web crawler is using pd.read_FilenameExtension.</font>
It can not only read local files but also read online files. 
Therefore you just need to follow following steps 
1. pd.read_FilenameExtension
2. data.to_FilenameExtension(file system)/data.to_sql(data base)
3. change path
4. for loop steps as mentioned above 

In [365]:
# import package
import pandas as pd
import time, datetime, os, glob, requests
from tqdm.notebook import tqdm
import tarfile
import gzip
import urllib.request
from urllib import parse
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# <font color="#26734d"> 【Exmple】</font>
* Website:[高公局歷史資料庫](https://tisvcloud.freeway.gov.tw/history-list.php)
* Data Name:最新消息(v2.0)
* File type:xml
* Output:File system
* Description:<br>
<img src="imgs/pd2.png" width="700" height="800">

In [322]:
# ----------------------------get url----------------------------------------
# data folder(url)
index_url = "https://tisvcloud.freeway.gov.tw/history/motc20/News/"
index = pd.read_html(index_url)[0]
# get data(url)
data_url = index_url + index["Name"][1]
# get data name(url)
data_name = pd.read_html(data_url)[0]["Name"]
data = pd.read_xml(data_url+data_name[0], compression="gzip", xpath='//*[contains(name(),"News")]')
# data cleaning
# 1.dropna(UpdateTime) 2.fillna(by first row) 3.drop useless col 4.reset index(drop original index)
data = data.dropna(subset=["UpdateTime"]).fillna(method='ffill').dropna(axis=1, how="all").reset_index(drop=True)

In [321]:
# ----------------------------data to file sys----------------------------------------
# create folder
index_foldername = r"C:\Users\88690\python\webcrawler\data\history\motc20\News"
if not os.path.exists(index_foldername):
    os.makedirs(index_foldername)

# get filename and path
filename = data_name[0].split(".")
filename = filename[0]+"."+filename[1]
filename = filename.replace("xml", "csv")
foldername = index_foldername + '\\' + index["Name"][0].split("/")[0]

# create folder
if not os.path.exists(foldername):
    os.mkdir(foldername)
    
# output data
fn = foldername + '\\' + filename
data.to_csv(fn, index=False)

# <font color="#26734d"> 【For Loop】</font>
* get 20 daily data to demo

In [351]:
# create folder
index_foldername = r"C:\Users\88690\python\webcrawler\data\history\motc20\News"
if not os.path.exists(index_foldername):
    os.makedirs(index_foldername)
# counter
count = 0
# ----------------------------get url----------------------------------------
# data folder(url)
index_url = "https://tisvcloud.freeway.gov.tw/history/motc20/News/"
index = pd.read_html(index_url)[0]
# get data(url)
for idn in tqdm(index["Name"][:2], desc='month:', colour='green'):
    print(f"----------------------------get {idn}----------------------------------------")
    data_url = index_url + idn
    # get data name(url)
    data_names = pd.read_html(data_url)[0]["Name"]
    # create folder    
    foldername = index_foldername + '\\' + idn.split("/")[0]
    if not os.path.exists(foldername):
        os.mkdir(foldername)
    # counter        
    count = count + len(data_names)    
    for data_name in tqdm(data_names[:20], desc='day:', colour='#006666'):
        data = pd.read_xml(data_url+data_name, compression="gzip", xpath='//*[contains(name(),"News")]')
        # data cleaning
        # 1.dropna(UpdateTime) 2.fillna(by first row) 3.drop useless col 4.reset index(drop original index)
        data = data.dropna(subset=["UpdateTime"]).fillna(method='ffill').dropna(axis=1, how="all").reset_index(drop=True)

        # get filename and path
        filename = data_name.split(".")
        filename = filename[0]+"."+filename[1]
        filename = filename.replace("xml", "csv")
        
        # ----------------------------data to file sys----------------------------------------
        print(f"----------------------------{filename} to file sys----------------------------------------",
             end="\r")

        # output data
        fn = foldername + '\\' + filename
        data.to_csv(fn, index=False)

print("")
print(f"totally get {count} data")

month::   0%|          | 0/2 [00:00<?, ?it/s]

----------------------------get 20230528/----------------------------------------


day::   0%|          | 0/20 [00:00<?, ?it/s]

----------------------------get 20230527/----------------------------------------------------


day::   0%|          | 0/20 [00:00<?, ?it/s]

----------------------------News_2340.csv to file sys----------------------------------------
totally get 1616 data


In [362]:
# read csv test
path = r"C:\Users\88690\python\webcrawler\data\history\motc20\News\20230527\News_2340.csv"
pd.read_csv(path).head()

,schemaLocation,UpdateTime,UpdateInterval,AuthorityCode,NewsID,Language,Title,NewsCategory,NewsURL,PublishTime
0,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:40:00+08:00,60.0,NFB,NaN,NaN,NaN,NaN,NaN,NaN
1,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230413150712855100021,Zh_tw,施工維護,4.0,https://www.freeway.gov.tw/default.aspx,2023-04-13T15:07:00+08:00
2,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230505055218092100041,Zh_tw,施工預告,4.0,https://www.freeway.gov.tw/default.aspx,2023-05-05T05:52:00+08:00
3,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230523195239598100043,Zh_tw,封閉預告,99.0,https://www.freeway.gov.tw/default.aspx,2023-05-23T19:52:00+08:00
4,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230527230012441100021,Zh_tw,施工維護,4.0,https://www.freeway.gov.tw/default.aspx,2023-05-27T23:00:00+08:00


In [371]:
# use glob module to cuple of csv
path = r"C:\Users\88690\python\webcrawler\data\history\motc20\News\20230527"
fnames = glob.glob(path + r'\*.csv', recursive=False)
print(len(fnames))

# get csv files in all folders(recursive)
path = r"C:\Users\88690\python\webcrawler\data\history\motc20\News"
fnames = glob.glob(path + r'\**\*.csv', recursive=True)
print(len(fnames))
pd.read_csv(fnames[0]).head()

20
43


,schemaLocation,UpdateTime,UpdateInterval,AuthorityCode,NewsID,Language,Title,NewsCategory,NewsURL,PublishTime
0,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:40:00+08:00,60.0,NFB,NaN,NaN,NaN,NaN,NaN,NaN
1,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230413150712855100021,Zh_tw,施工維護,4.0,https://www.freeway.gov.tw/default.aspx,2023-04-13T15:07:00+08:00
2,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230505055218092100041,Zh_tw,施工預告,4.0,https://www.freeway.gov.tw/default.aspx,2023-05-05T05:52:00+08:00
3,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230523195239598100043,Zh_tw,封閉預告,99.0,https://www.freeway.gov.tw/default.aspx,2023-05-23T19:52:00+08:00
4,http://traffic.transportdata.tw/standard/traff...,2023-05-27T23:39:00+08:00,60.0,NFB,20230527230012441100021,Zh_tw,施工維護,4.0,https://www.freeway.gov.tw/default.aspx,2023-05-27T23:00:00+08:00


# <font color="#26734d"> 【Exmple】</font>
* Website:[高公局歷史資料庫](https://tisvcloud.freeway.gov.tw/history-list.php)
* Data Name:各類車種旅次數量 (M08A)
* File type:csv
* Output:File system
* Description:<br>
<img src="imgs/pd1.png" width="500" height="600">


In [377]:
colname = ["TimeInterval", "GantryFrom", "GantryTo", "VehicleType", "Traffic"]

In [493]:
index_foldername = r"C:\Users\88690\python\webcrawler\data\history\TDCS\M08A"
url = "https://tisvcloud.freeway.gov.tw/history/TDCS/M08A/"
index_Alldata = pd.read_html(url)[0]
dates = index_Alldata["Name"]
for date in tqdm(dates[34:37], desc='day:', colour='green'):
    if not "gz" in date:
        print(f"-------------get {date}----------------", end='\r')
        path = index_foldername+'\\'+date.split('/')[0]
        if not os.path.exists(path):
            os.makedirs(path)

        index_data = pd.read_html(url+date)[0]
        hrs = index_data["Name"]
        for hr in tqdm(hrs[:2], desc='hr:', colour='#006666', leave=False):
            # print(f"-------------get {hr}---------------", end='\r')
            path_ = path+'\\'+hr.split('/')[0]
            if not os.path.exists(path_):
                os.makedirs(path_)
            
            data_names = pd.read_html(url+date+hr)[0]["Name"]
            for data_name in tqdm(data_names, desc='min:', colour='#336699', leave=False):
                # print(f"-------------get {data_name}---------------", end='\r')
                url_ = url+date+hr
                data = pd.read_csv(url_+data_name, names=colname)
                fn = path_+'\\'+data_name
                data.to_csv(fn, index=False)
    else:
        print(f"-------------get {date}----------------", end='\r')
        # URL of the tar.gz file
        url = f'https://tisvcloud.freeway.gov.tw/history/TDCS/M08A/{date}'
        # Download the tar.gz file
        urllib.request.urlretrieve(url, 'example.tar.gz')

        path = r"C:\Users\88690\python\webcrawler\data\history\TDCS"
        # Open the downloaded tar.gz file in read mode
        with tarfile.open('example.tar.gz', 'r:*') as tar:
            # Extract all files in the archive
            tar.extractall(path)

day::   0%|          | 0/3 [00:00<?, ?it/s]

In [489]:
# URL of the tar.gz file
url = f'https://tisvcloud.freeway.gov.tw/history/TDCS/M08A/{dates[35]}'
# Download the tar.gz file
urllib.request.urlretrieve(url, 'example.tar.gz')

path = r"C:\Users\88690\python\webcrawler\data\history\TDCS"
# Open the downloaded tar.gz file in read mode
with tarfile.open('example.tar.gz', 'r:*') as tar:
    # Extract all files in the archive
    tar.extractall(path)

# column name of csv file is incorrect 
# -> pd.read_csv(FileName, <"name=colname">)

In [492]:
# read csv test
test = r"C:\Users\88690\python\webcrawler\M08A\20230415\00\TDCS_M08A_20230415_000000.csv"
pd.read_csv(test, names=colname).head()

,TimeInterval,GantryFrom,GantryTo,VehicleType,Traffic
0,2023-04-15 00:00,01F0005N,01F0005N,31,3
1,2023-04-15 00:00,01F0005N,01F0005N,32,2
2,2023-04-15 00:00,01F0005N,01F0005N,5,1
3,2023-04-15 00:00,01F0005S,01F0005S,31,13
4,2023-04-15 00:00,01F0005S,01F0005S,41,1


In [360]:
# set html css
%%html
<style>
table{
    float:left;
}
</style>

# <font color="#26734d"> 【Exmple】</font>
* Website:[觀測資料查詢](https://e-service.cwb.gov.tw/HistoryDataQuery/)
* Data Name:天氣報表(年/月/日)
* File type:html/csv
* Output:File system
* Description:
    * [README](https://e-service.cwb.gov.tw/HistoryDataQuery/downloads/Readme.pdf)
    *[現存測站](https://e-service.cwb.gov.tw/wdps/obs/state.htm)


In [535]:
url = "https://e-service.cwb.gov.tw/wdps/obs/state.htm"
station_data = pd.read_html(url)[0]

In [643]:
station_data.to_csv("station_data.csv", index=False)
station_data.head()

In [549]:
datatype = "Day"
stids = station_data["站號"]
stnames = [parse.quote(st, encoding='utf-8').replace("%", "%25") for st in station_data["站名"]]
alts = round(station_data["海拔高度(m)"], 1)
date = "2023-05-01"

In [556]:
url = f"https://e-service.cwb.gov.tw/HistoryDataQuery/{datatype}DataController.do?command=viewMain&station={stids[2]}&stname={stnames[2]}&datepicker={date}&altitude={alts[2]}m"
resp = requests.get(url)

In [557]:
display(pd.read_html(resp.text)[0])
pd.read_html(resp.text)[1].head()

,0,1,2,3,4,5,6,7,8,9
0,日報表 (daily data),測站:466900_淡水,466881_新北 466900_淡水 C0A520_山佳 C0A530_坪林 C0...,NaN,觀測時間:2023-05-01,NaN,NaN,CSV下載,NaN,資料定義請詳見 ◎網頁說明Readme


Unnamed: 0_level_0     Press            temperature    dew point      RH  \
          觀測時間(hour) 測站氣壓(hPa) 海平面氣壓(hPa)       氣溫(℃)      露點溫度(℃) 相對溼度(%)   
             ObsTime   StnPres    SeaPres Temperature Td dew point      RH   
0                  1    1013.2     1015.6        18.6         17.1      91   
1                  2    1012.6     1015.0        18.8         16.4      86   
2                  3    1012.2     1014.6        18.6         17.4      93   
3                  4    1012.1     1014.5        18.5         17.2      92   
4                  5    1012.4     1014.8        18.2         17.2      94   

    WD/WS                                             Precp            \
  風速(m/s) 風向(360degree) 最大陣風(m/s) 最大陣風風向(360degree) 降水量(mm)   降水時數(h)   
       WS            WD    WSGust            WDGust   Precp PrecpHour   
0     0.6            50       2.8                20     0.0       0.0   
1     1.2            30       2.6                10     0.0       0.0   
2     0.4           340       1.8                40     0.0       0.0   
3     0.3           130       1.2                30     0.0       0.0   
4     0.8            90       1.6                90     0.0       0.0   

  SunShine              visibility   UVI        Cloud  
   日照時數(h) 全天空日射量(MJ/㎡)    能見度(km) 紫外線指數    總雲量(0~10)  
  SunShine     GloblRad       Visb   UVI Cloud Amount  
0      ...          0.0        ...     0          ...  
1      ...          0.0        ...     0          ...  
2      ...          0.0        ...     0          ...  
3      ...          0.0        ...     0          ...  
4      ...          0.0        ...     0          ...

# <font color="#26734d"> 【Set Up Session】</font>

In [570]:
# dictionary for station name
station_dict = {}
for i in range(len(stnames)):
    station_disc.update({station_data["站名"][i]:[stids[i], stnames[i], alts[i]]})
# station_disc.get("淡水")

In [513]:
# set up session
sess = requests.session()
url = "https://e-service.cwb.gov.tw/HistoryDataQuery/"
resp = sess.get(url)

In [582]:
url = "https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466900&stname=%25E6%25B7%25A1%25E6%25B0%25B4&datepicker=2022-05&altitude=19.0m"
resp = sess.get(url)

In [583]:
"goErrorPage()" in resp.text

False

In [584]:
pd.read_html(resp.text)[1].head()

Unnamed: 0_level_0     press                                           \
           觀測時間(day) 測站氣壓(hPa) 海平面氣壓(hPa) 測站最高氣壓(hPa)     測站最高氣壓時間(LST)   
             ObsTime   StnPres    SeaPres  StnPresMax    StnPresMaxTime   
0                  1    1013.8     1016.2      1016.1  2022-05-01 21:48   
1                  2    1014.7     1017.1      1016.3  2022-05-02 21:03   
2                  3    1014.5     1016.9      1015.7  2022-05-03 00:03   
3                  4    1011.6     1014.0      1014.2  2022-05-04 00:01   
4                  5    1009.8     1012.1      1010.7  2022-05-05 09:04   

                                temperature                            ...  \
  測站最低氣壓(hPa)     測站最低氣壓時間(LST)       氣溫(℃) 最高氣溫(℃)       最高氣溫時間(LST)  ...   
   StnPresMin    StnPresMinTime Temperature   T Max        T Max Time  ...   
0      1011.8  2022-05-01 02:54        17.7    20.0  2022-05-01 00:02  ...   
1      1013.4  2022-05-02 14:04        15.4    16.9  2022-05-02 12:27  ...   
2      1013.7  2022-05-03 14:22        19.9    22.7  2022-05-03 13:49  ...   
3      1010.0  2022-05-04 17:14        23.8    27.3  2022-05-04 13:18  ...   
4      1008.6  2022-05-05 02:47        22.9    24.8  2022-05-05 10:27  ...   

          Precp                      SunShine                            \
  最大六十分鐘降水量(mm) 最大六十分鐘降水量起始時間(LST) 日照時數(hour)       日照率(%) 全天空日射量(MJ/㎡)   
     PrecpMax60     PrecpMax60Time   SunShine SunShineRate     GloblRad   
0           3.5   2022-05-01 15:18        0.0          0.0         3.22   
1           1.5   2022-05-02 03:06        0.0          0.0         5.34   
2             T   2022-05-03 00:01        0.0          0.0         6.03   
3             T   2022-05-04 04:43        2.1         16.0        18.03   
4           0.0                ...        0.8          6.1        13.75   

  visibility Evaperation      UVI                        Cloud  
     能見度(km)   A型蒸發量(mm) 日最高紫外線指數 日最高紫外線指數時間(LST)    總雲量(0~10)  
    VisbMean       EvapA  UVI Max    UVI Max Time Cloud Amount  
0        ...         ...      ...             ...          ...  
1        ...         ...      ...             ...          ...  
2        ...         ...      ...             ...          ...  
3        ...         ...      ...             ...          ...  
4        ...         ...      ...             ...          ...  

[5 rows x 35 columns]

In [534]:
sess.close()

# <font color="#26734d"> 【Class(OOP】</font>

In [716]:
class Weather:
    def __init__(self):
        # self._url = "https://e-service.cwb.gov.tw/wdps/obs/state.htm"
        self._station_data = pd.read_csv("station_data.csv")
        self._station_dict = {}
        stids = station_data["站號"]
        stnames = [parse.quote(st, encoding='utf-8').replace("%", "%25") for st in self._station_data["站名"]]
        alts = round(self._station_data["海拔高度(m)"], 1)
        for i in range(len(stnames)):
            self._station_dict.update({self._station_data["站名"][i]:[stids[i], stnames[i], alts[i]]})
    
        self._date_format = {'Year':'%Y',
                             'Month':'%Y-%m',
                             'Day':'%Y-%m-%d'}
        
        self._session_flag = False
        # self._data = []
        self._datatype = 'Month'
    
    def setup_session(self):
        self._session = requests.session()
        url = "https://e-service.cwb.gov.tw/HistoryDataQuery/"
        resp = self._session.get(url)
        if resp.ok:
            self._session_flag = True
            return "Set up Connection!"
        else:
            return "Set up Connection failly!"
        
    def close_session(self):
        self._session.close()
        return "Connection is closed!"
    
    # def is_valid_date(self, date_str, date_format):
    #     try:
    #         datetime.strptime(date_str, date_format)
    #         return True
    #     except ValueError:
    #         return False
    def date_check(self, date):
        date_format = self._date_format.get(self._datatype)
        try:
            date = datetime.strptime(date, date_format)
            return date.strftime(date_format)
        except ValueError:
            return False
        
    def generate_date_range(self, start_date, end_date):
        date_format = self._date_format.get(self._datatype)
        try:
            start_date = datetime.strptime(start_date, date_format)
            end_date = datetime.strptime(end_date, date_format)
            date_range = []
            current_date = start_date
            while current_date <= end_date:
                date_range.append(current_date.strftime(date_format))
                if self._datatype == 'Year':
                    current_date += relativedelta(years=1)
                elif self._datatype == 'Month':
                    current_date += relativedelta(months=1)
                else:
                    current_date += relativedelta(days=1)
            return date_range
        
        except ValueError:
            return "Date format is wrong"
        
    def get_data(self, stname, date):
        date = self.date_check(date)
        if date == False:
            return "Date format is wrong"
        
        if not self._session_flag:
            return "Connecion hasn't set up"
        else:
            key = self._station_dict.get(stname)
            self._url = f"https://e-service.cwb.gov.tw/HistoryDataQuery/{self._datatype}DataController.do?command=viewMain&station={key[0]}&stname={key[1]}&datepicker={date}&altitude={key[2]}m"
            resp = self._session.get(self._url)
            if "goErrorPage()" in resp.text:
                return "Go Error Page!"
            else:
                self._data = pd.read_html(resp.text)
                return self._data
 
    def get_datatype(self):
        return self._datatype
    
    def set_datatype(self, datatype):
        self._datatype = datatype
    
    def get_stname(self):
        return self._station_data["站名"]
    
    def get_url(self):
        return self._url

In [717]:
weather = Weather()

In [688]:
weather.get_stname()[0]

'五分山雷達站'

In [718]:
weather.setup_session()
weather.set_datatype("Month")

In [720]:
weather.get_data("淡水", "2023-05")[1].head()

Unnamed: 0_level_0     press                                           \
           觀測時間(day) 測站氣壓(hPa) 海平面氣壓(hPa) 測站最高氣壓(hPa)     測站最高氣壓時間(LST)   
             ObsTime   StnPres    SeaPres  StnPresMax    StnPresMaxTime   
0                  1    1013.3     1015.6      1014.5  2023-05-01 09:15   
1                  2    1013.2     1015.5      1014.6  2023-05-02 08:26   
2                  3    1011.7     1014.0      1013.5  2023-05-03 07:58   
3                  4    1008.1     1010.4      1010.7  2023-05-04 00:01   
4                  5    1003.9     1006.2      1006.1  2023-05-05 00:01   

                                temperature                            ...  \
  測站最低氣壓(hPa)     測站最低氣壓時間(LST)       氣溫(℃) 最高氣溫(℃)       最高氣溫時間(LST)  ...   
   StnPresMin    StnPresMinTime Temperature   T Max        T Max Time  ...   
0      1012.1  2023-05-01 03:23        22.6    27.8  2023-05-01 12:20  ...   
1      1011.7  2023-05-02 16:15        26.9    32.0  2023-05-02 13:21  ...   
2      1010.3  2023-05-03 17:02        25.1    28.1  2023-05-03 11:44  ...   
3      1006.0  2023-05-04 23:08        25.7    29.5  2023-05-04 14:50  ...   
4      1001.6  2023-05-05 18:17        28.0    31.7  2023-05-05 11:25  ...   

          Precp                      SunShine                            \
  最大六十分鐘降水量(mm) 最大六十分鐘降水量起始時間(LST) 日照時數(hour)       日照率(%) 全天空日射量(MJ/㎡)   
     PrecpMax60     PrecpMax60Time   SunShine SunShineRate     GloblRad   
0           0.0                ...        9.2         70.4        24.66   
1           0.0                ...        8.4         64.2        23.32   
2           0.0                ...        7.3         55.7        21.14   
3           0.0                ...        8.2         62.5        24.34   
4           0.0                ...        7.3         55.5        22.55   

  visibility Evaperation      UVI                          Cloud  
     能見度(km)   A型蒸發量(mm) 日最高紫外線指數   日最高紫外線指數時間(LST)    總雲量(0~10)  
    VisbMean       EvapA  UVI Max      UVI Max Time Cloud Amount  
0        ...         ...       10  2023-05-01 12:00          ...  
1        ...         ...       11  2023-05-02 12:00          ...  
2        ...         ...       11  2023-05-03 12:00          ...  
3        ...         ...       11  2023-05-04 12:00          ...  
4        ...         ...       11  2023-05-05 13:00          ...  

[5 rows x 35 columns]

In [730]:
dates = weather.generate_date_range("2023-01", "2023-05")
data_lst = [weather.get_data("淡水", date) for date in dates]
data_infos = [data_info[0] for data_info in data_lst]
datas = [data[1] for data in data_lst]

In [748]:
# display(data_infos[0])
print(data_infos[0].iat[0, 1])
print(data_infos[0].iat[0, 4])

測站:466900_淡水
觀測時間:2023-01


In [735]:
datas[0].head()

Unnamed: 0_level_0     press                                           \
           觀測時間(day) 測站氣壓(hPa) 海平面氣壓(hPa) 測站最高氣壓(hPa)     測站最高氣壓時間(LST)   
             ObsTime   StnPres    SeaPres  StnPresMax    StnPresMaxTime   
0                  1    1021.2     1023.6      1022.8  2023-01-01 09:24   
1                  2    1021.5     1023.9      1023.3  2023-01-02 21:08   
2                  3    1022.8     1025.2      1024.2  2023-01-03 08:47   
3                  4    1022.4     1024.8      1024.8  2023-01-04 09:07   
4                  5    1021.2     1023.6      1023.2  2023-01-05 08:38   

                                temperature                            ...  \
  測站最低氣壓(hPa)     測站最低氣壓時間(LST)       氣溫(℃) 最高氣溫(℃)       最高氣溫時間(LST)  ...   
   StnPresMin    StnPresMinTime Temperature   T Max        T Max Time  ...   
0      1019.2  2023-01-01 14:08        19.7    22.6  2023-01-01 13:02  ...   
1      1019.6  2023-01-02 14:07        18.0    19.5  2023-01-02 10:52  ...   
2      1021.2  2023-01-03 04:35        15.6    16.7  2023-01-03 07:39  ...   
3      1020.4  2023-01-04 15:27        16.5    18.7  2023-01-04 13:56  ...   
4      1019.5  2023-01-05 14:40        18.9    23.0  2023-01-05 12:45  ...   

          Precp                      SunShine                            \
  最大六十分鐘降水量(mm) 最大六十分鐘降水量起始時間(LST) 日照時數(hour)       日照率(%) 全天空日射量(MJ/㎡)   
     PrecpMax60     PrecpMax60Time   SunShine SunShineRate     GloblRad   
0             T   2023-01-01 00:01        0.5          4.7         8.71   
1           0.5   2023-01-02 02:05        0.0          0.0         2.19   
2             T   2023-01-03 00:01        0.0          0.0         4.83   
3             T   2023-01-04 00:03        0.3          2.8         5.92   
4             T   2023-01-05 03:02        3.5         33.0        11.58   

  visibility Evaperation      UVI                          Cloud  
     能見度(km)   A型蒸發量(mm) 日最高紫外線指數   日最高紫外線指數時間(LST)    總雲量(0~10)  
    VisbMean       EvapA  UVI Max      UVI Max Time Cloud Amount  
0        ...         ...        5  2023-01-01 13:00          ...  
1        ...         ...        1  2023-01-02 11:00          ...  
2        ...         ...        3  2023-01-03 12:00          ...  
3        ...         ...        3  2023-01-04 14:00          ...  
4        ...         ...        6  2023-01-05 13:00          ...  

[5 rows x 35 columns]

In [ ]:
data_lst[0]